In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
df = pd.read_csv('spiralData.csv')
points = df.loc[:,['X','Y']]
labels = df.loc[:,['Class']]

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [5]:
trainData = df_shuffled.loc[:599,['X','Y']]
trainLabel = df_shuffled.loc[:599,['Class']]
valData = df_shuffled.loc[600:799,['X','Y']]
valLabel = df_shuffled.loc[600:799,['Class']]
testData = df_shuffled.loc[800:,['X','Y']]
testLabel = df_shuffled.loc[800:,['Class']]

In [6]:
# init parameters
training_rate = 0.01
training_epochs = 300
batch_size = 100
l2_reg = 0.003

n_input = 2    # input dimension
n_classes = 3  # output dimension
n_hidden = 100 # hidden layer neurons

In [7]:
# construct each layer
points = tf.placeholder(tf.float32, [None, n_input])
label = tf.placeholder(tf.float32, [None, n_classes])

# first hidden layer
W1 = tf.Variable(tf.random_normal([n_input, n_hidden]))
b1 = tf.Variable(tf.random_normal([n_hidden]))

# output layer
W2 = tf.Variable(tf.random_normal([n_hidden, n_classes]))
b2 = tf.Variable(tf.random_normal([n_classes]))

In [8]:
# construct whole neural network
hidden_layer = tf.nn.relu(tf.add(tf.matmul(points, W1), b1)) # points * W + b
output_layer = tf.matmul(hidden_layer, W2) + b2

In [9]:
# setup loss and optimizer
softmax_result = tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=output_layer)
l2_loss = l2_reg*tf.nn.l2_loss(W1) + l2_reg*tf.nn.l2_loss(b1) + l2_reg*tf.nn.l2_loss(W1) + l2_reg*tf.nn.l2_loss(b2)
cross_entropy = tf.reduce_mean(softmax_result)
optimizer = tf.train.GradientDescentOptimizer(training_rate)
train_step = optimizer.minimize(cross_entropy)

In [10]:
init = tf.global_variables_initializer()

In [11]:
length_tmp = len(trainLabel)
trainLabel_onehot = np.zeros((length_tmp,n_classes))
trainLabel_onehot[range(length_tmp),list(trainLabel.transpose().values)] = 1

length_tmp = len(testLabel)
testLabel_onehot = np.zeros((length_tmp,n_classes))
testLabel_onehot[range(length_tmp),list(testLabel.transpose().values)] = 1

length_tmp = len(valLabel)
valLabel_onehot = np.zeros((length_tmp,n_classes))
valLabel_onehot[range(length_tmp),list(valLabel.transpose().values)] = 1

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(trainData/batch_size))
        
        # Loop over all batches
        for i in range(total_batch):
            offset = 0
            batch_x = trainData[offset:offset+batch_size]
            batch_y = trainLabel_onehot[offset:offset+batch_size]
            offset += batch_size
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_step, cross_entropy],
                            feed_dict={points: batch_x, label: batch_y})
            #print("D.chaos cost {}/{}".format(c,avg_cost))
        
            # Compute average loss
            avg_cost += c / total_batch
        
        # accuracy on validation data
        correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        # Display logs per epoch step
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))   
        print("Accuracy on training data: %4f" % accuracy.eval({points: trainData, label: trainLabel_onehot}),end=' ')
        print(", on validation data: %4f" % accuracy.eval({points: valData, label: valLabel_onehot}))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(label, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy on test data: %4f" % accuracy.eval({points: testData, label: testLabel_onehot}))

Epoch: 0001 cost= 0.503881791
Accuracy on training data: 0.861667 , on validation data: 0.885000
Epoch: 0002 cost= 0.320240317
Accuracy on training data: 0.870000 , on validation data: 0.885000
Epoch: 0003 cost= 0.280690902
Accuracy on training data: 0.878333 , on validation data: 0.890000
Epoch: 0004 cost= 0.255926906
Accuracy on training data: 0.895000 , on validation data: 0.900000
Epoch: 0005 cost= 0.238148958
Accuracy on training data: 0.908333 , on validation data: 0.915000
Epoch: 0006 cost= 0.224098776
Accuracy on training data: 0.925000 , on validation data: 0.915000
Epoch: 0007 cost= 0.212366357
Accuracy on training data: 0.931667 , on validation data: 0.935000
Epoch: 0008 cost= 0.202025089
Accuracy on training data: 0.933333 , on validation data: 0.935000
Epoch: 0009 cost= 0.192826823
Accuracy on training data: 0.936667 , on validation data: 0.945000
Epoch: 0010 cost= 0.185145712
Accuracy on training data: 0.938333 , on validation data: 0.950000
Epoch: 0011 cost= 0.178280919


Epoch: 0086 cost= 0.060821126
Accuracy on training data: 0.971667 , on validation data: 0.995000
Epoch: 0087 cost= 0.060341287
Accuracy on training data: 0.971667 , on validation data: 0.995000
Epoch: 0088 cost= 0.059866770
Accuracy on training data: 0.971667 , on validation data: 0.995000
Epoch: 0089 cost= 0.059398908
Accuracy on training data: 0.971667 , on validation data: 0.995000
Epoch: 0090 cost= 0.058937897
Accuracy on training data: 0.971667 , on validation data: 0.995000
Epoch: 0091 cost= 0.058483120
Accuracy on training data: 0.973333 , on validation data: 0.995000
Epoch: 0092 cost= 0.058035059
Accuracy on training data: 0.973333 , on validation data: 0.995000
Epoch: 0093 cost= 0.057593606
Accuracy on training data: 0.973333 , on validation data: 0.995000
Epoch: 0094 cost= 0.057158628
Accuracy on training data: 0.973333 , on validation data: 0.995000
Epoch: 0095 cost= 0.056729967
Accuracy on training data: 0.973333 , on validation data: 0.995000
Epoch: 0096 cost= 0.056307481


Epoch: 0171 cost= 0.036118065
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0172 cost= 0.035947612
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0173 cost= 0.035778619
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0174 cost= 0.035611186
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0175 cost= 0.035445647
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0176 cost= 0.035281789
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0177 cost= 0.035119230
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0178 cost= 0.034958300
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0179 cost= 0.034798907
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0180 cost= 0.034640911
Accuracy on training data: 0.976667 , on validation data: 0.995000
Epoch: 0181 cost= 0.034484665


Epoch: 0256 cost= 0.025797785
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0257 cost= 0.025712452
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0258 cost= 0.025627672
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0259 cost= 0.025543350
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0260 cost= 0.025459526
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0261 cost= 0.025376253
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0262 cost= 0.025293434
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0263 cost= 0.025211194
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0264 cost= 0.025129548
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0265 cost= 0.025048432
Accuracy on training data: 0.978333 , on validation data: 1.000000
Epoch: 0266 cost= 0.024967824
